# AWS ParallelCluster Programmatic Configuration

Investigate the pcluster configure argument 

In [98]:
import pcluster
import pcluster.configure.easyconfig as easyconfig
import requests
import random

from pcluster.cluster_model import ClusterModel
from pcluster.config.hit_converter import HitConverter
from pcluster.config.pcluster_config import PclusterConfig
from pcluster.config.validators import HEAD_NODE_UNSUPPORTED_INSTANCE_TYPES, HEAD_NODE_UNSUPPORTED_MESSAGE
from pcluster.configure.networking import (
    NetworkConfiguration,
    PublicPrivateNetworkConfig,
    automate_subnet_creation,
    automate_vpc_with_subnet_creation,
)
from pcluster.configure.utils import get_regions, get_resource_tag, handle_client_exception, prompt, prompt_iterable
from pcluster.utils import (
    error,
    get_default_instance_type,
    get_region,
    get_supported_az_for_multi_instance_types,
    get_supported_az_for_one_instance_type,
    get_supported_compute_instance_types,
    get_supported_instance_types,
    get_supported_os_for_scheduler,
    get_supported_schedulers,
)
from pcluster.configure.easyconfig import ClusterConfigureHelper

import json
import logging
import os
import sys
from collections import OrderedDict
import boto3
import tempfile
import string

from datetime import datetime
from jinja2 import Environment, BaseLoader

#ID = datetime.today().strftime('%Y%m%d')
ID = '20210127'

## Create A Project ID

Create a random project ID for the state, buckets, etc.

In [5]:
#ID = datetime.today().strftime('%Y%m%d')
ID = '20210127'

N = 6
random_string = ''.join(random.choices(string.ascii_lowercase + string.digits, k=N))
random_string

'cjwow221'

Before we get to to this point we should have:
    
    * Terraform State 
    
    Then with terraform we create the following: 
    
    * S3 Bucket Installation
    * S3 Bucket User Admin
    * EC2 Keypair
    * Custom AMI 
    * Additional Security Group

This is what the pcluster file ends up looking like:

```
[aws]
aws_region_name = {{cookiecutter.aws_region}}

[aliases]
ssh = ssh {CFN_USER}@{MASTER_IP} {ARGS}

[global]
cluster_template = default
update_check = true
sanity_check = true

[cluster default]
key_name = slurm-2.10.0-us-east-1
base_os = alinux2
scheduler = slurm
master_instance_type = t3a.medium
vpc_settings = default
queue_settings = compute,extralarge
custom_ami = {{cookiecutter.custom_ami_id}}

s3_read_resource = arn:aws:s3:::{{cookiecutter.installation_bucket}}/*
post_install = s3://{{cookiecutter.installation_bucket}}/install_all_the_things.sh

[vpc default]
vpc_id = {{cookiecutter.vpc_id}}
master_subnet_id = {{cookiecutter.master_subnet_id}}
compute_subnet_id = {{cookiecutter.compute_subnet_id}}
use_public_ips = false
additional_sg = {{cookiecutter.additional_security_group}}

[queue compute]
enable_efa = false
enable_efa_gdr = false
compute_resource_settings = littlemem,bigmem,mediummem

[compute_resource littlemem]
instance_type = t3a.medium
max_count = 1000

[compute_resource mediummem]
instance_type = t3a.large
max_count = 1000

[compute_resource bigmem]
instance_type = t3.2xlarge
max_count = 1000

[queue extralarge]
enable_efa = false
enable_efa_gdr = false
compute_resource_settings = extralarge

[compute_resource extralarge]
instance_type = m4.10xlarge
max_count = 1000
```

In [100]:
pcluster_template = """[aws]
aws_region_name = {{cookiecutter.aws_region}}

[aliases]
ssh = ssh {CFN_USER}@{MASTER_IP} {ARGS}

[global]
cluster_template = default
update_check = true
sanity_check = true

[cluster default]
key_name = slurm-2.10.0-us-east-1
base_os = alinux2
scheduler = slurm
master_instance_type = t3a.medium
vpc_settings = default
queue_settings = compute,extralarge
custom_ami = {{cookiecutter.custom_ami_id}}

s3_read_resource = arn:aws:s3:::{{cookiecutter.s3_installation_bucket}}/*
post_install = s3://{{cookiecutter.s3_installation_bucket}}/install_all_the_things.sh

[vpc default]
vpc_id = {{cookiecutter.vpc_id}}
master_subnet_id = {{cookiecutter.master_subnet_id}}
compute_subnet_id = {{cookiecutter.compute_subnet_id}}
use_public_ips = false
additional_sg = {{cookiecutter.additional_security_group_id}}

{% for queue in cookiecutter.queues %}
[queue {{queue.name}}]
enable_efa = false
enable_efa_gdr = false
compute_resource_settings = {{queue.compute_resource_settings}}
{% endfor %}

{% for compute_resource in cookiecutter.compute_resources %}
[compute_resource {{compute_resource.name}}]
instance_type = {{compute_resource.instance_type}}
min_count = {{compute_resource.min_count}}  
max_count = {{compute_resource.max_count}}      
{% endfor %}
"""

In [33]:
PCLUSTER_VERSION='2.10.1'
AMIS_LIST='https://raw.githubusercontent.com/aws/aws-parallelcluster/v{version}/amis.txt'.format(version=PCLUSTER_VERSION)

In [204]:
tconfig = tempfile.NamedTemporaryFile(delete=False)
tconfig.name

'/tmp/tmpeww20hi5'

In [205]:
pcluster_config = PclusterConfig(config_file=tconfig.name, fail_on_error=False, auto_refresh=False)

In [84]:
# These are all variables that will be passed in and supplied through cookiecutter
# We'll also grab the terraform output
# We'll use pcluster to create the subnets because it's very picky about subnets

CONFIG = {
    'id': ID,
    'hosted_zone_id': '',
    'project': 'slurm-cluster',
    'stage': 'development',
    'vpc_id': 'vpc-2658435c',
    'cloudformation_stack': 'parallelcluster-{{cookiecutter.project}}-{{cookiecutter.id}}-{{cookiecutter.stage}}',
    # These all get read in from terraform
    's3_installation_bucket': "{{cookiecutter.project}}-{{cookiecutter.id}}-{{cookiecutter.stage}}-installation",
    's3_user_data_bucket': "{{cookiecutter.project}}-{{cookiecutter.id}}-{{cookiecutter.stage}}-user-data",
    's3_admin_bucket': "{{cookiecutter.project}}-{{cookiecutter.id}}-{{cookiecutter.stage}}-admin",
    'aws_region': 'us-east-1',
    'tags': {
        'Name': '{{cookiecutter.project}}-{{cookiecutter.id}}-{{cookiecutter.stage}}',
        'Project': '{{cookiecutter.project}}',
        'Stage': '{{cookiecutter.stage}}',

    },
    # Terraform recipe vars
    'terraform_recipes': {
        # Bootstrap the state
        'terraform_state': 'terraform-state',
        # Supply the resources, build the custom AMI
        'pcluster_resources': 'pcluster-resources',
        # After install all the apps - easybuild, modules, etc
        'pcluster_apps': 'pcluster-apps',
    },
    # Terraform state Vars
    # these end up looking different because the module tacks on some names
    "terraform_state": {
        "s3_bucket": "{{cookiecutter.project}}-{{cookiecutter.id}}",
        "s3_bucket_full_name": "{{cookiecutter.project}}-{{cookiecutter.id}}-{{cookiecutter.stage}}-terraform-state",
        "dynamo_db_table": "{{cookiecutter.project}}-{{cookiecutter.id}}-{{cookiecutter.stage}}-terraform-state-lock",
    },
    # Store the terraform output
    'terraform_output': {
        'terraform_state': {},
        'pcluster_resources': {},
        'pcluster_apps': {},
    },
    # PCluster Vars
    'pcluster': {
        'master_instance_type': 't3a.2xlarge',
        'scheduler': 'slurm',
        'base_os': 'alinux2',
        'pcluster_version': '2.10.1',
        'key_pair': '',
        'min_cluster_size': 0,
        'max_cluster_size': 100,
        'head_node_instance_type': 't3a.2xlarge',
        # This doesn't matter because we're using queues
        # But still used
        'compute_node_instance_type': 't3a.2xlarge',
        'master_subnet_id': '',
        'compute_subnet_id': '',
        'compute_resources': [
            {
                'instance_type': 't3a.medium',
                'min_count': 0,
                'max_count': 100,
            },
            {
                'instance_type': 't3a.large',
                'min_count': 0,
                'max_count': 100,
            },
            {
                'instance_type': 't3a.2xlarge',
                'min_count': 0,
                'max_count': 100,
            },
            {
                'instance_type': 'm4.large',
                'min_count': 0,
                'max_count': 100,
            },
            {
                'instance_type': 'm4.xlarge',
                'min_count': 0,
                'max_count': 100,
            },
            {
                'instance_type': 'm4.2xlarge',
                'min_count': 0,
                'max_count': 100,
            },
            {
                'instance_type': 'g4dn.xlarge',
                'min_count': 0,
                'max_count': 100,
            },
            {
                'instance_type': 'g4dn.2xlarge',
                'min_count': 0,
                'max_count': 100,
            },
            {
                'instance_type': 'g4dn.4xlarge',
                'min_count': 0,
                'max_count': 100,
            },
        ],
        'queues': [
            {
                'name': 'dev',
                'enable_efa': False,
                'enable_efa_gdr': False,
                'compute_resource_instance_types': ['t3a.medium', 't3a.large', 't3a.2xlarge'],
            },
            {
                'name': 'cpu',
                'enable_efa': False,
                'enable_efa_gdr': False,
                'compute_resource_instance_types': ['m4.large', 'm4.xlarge', 'm4.2xlarge'],
            },
            {
                'name': 'gpu',
                'enable_efa': False,
                'enable_efa_gdr': False,
                'compute_resource_instance_types': ['g4dn.xlarge', 'g4dn.2xlarge', 'g4dn.4xlarge'],
            },
        ],
        'queue_settings': '',
        'efs_resources': [
            {
                "name": "apps",
                "efs_id": False,
                "performance_mode": "generalPurpose"
            },
            {
                "name": "scratch",
                "efs_id": False,
                "performance_mode": "maxIO"
            }
        ],
    }
}


BASE_DIR='/home/jovyan/TEST_APP/'
with open(os.path.join(BASE_DIR, 'cookiecutter.json)', 'w') as outfile:
    json.dump(CONFIG, outfile, indent=4)    

with open('/home/jovyan/app/aws_parallelcluster_wrapper/_cookiecutter_templates/terraform-modules/pcluster-resources/{{cookiecutter.terraform_recipes.pcluster_resources}}/cookiecutter.json', 'w') as outfile:
    json.dump(CONFIG, outfile, indent=4)
    
with open('/home/jovyan/app/aws_parallelcluster_wrapper/_cookiecutter_templates/terraform-modules/pcluster-resources/cookiecutter.json', 'w') as outfile:
    json.dump(CONFIG, outfile, indent=4)
    
with open('/home/jovyan/app/aws_parallelcluster_wrapper/_cookiecutter_templates/terraform-modules/pcluster-apps/cookiecutter.json', 'w') as outfile:
    json.dump(CONFIG, outfile, indent=4)
      

## PConfig Cluster Section

In [191]:
cluster_section = pcluster_config.get_section("cluster")

global_config = pcluster_config.get_section("global")
cluster_label = global_config.get_param_value("cluster_template")

vpc_section = pcluster_config.get_section("vpc")
vpc_label = vpc_section.label
vpc_parameters = {}

## Set Variables from our Config

In [131]:
cluster_config = ClusterConfigureHelper(cluster_section, CONFIG['scheduler'])
cluster_config.max_cluster_size = CONFIG['max_cluster_size']
cluster_config.head_node_instance_type = CONFIG['head_node_instance_type']
cluster_config.compute_node_instance_type = CONFIG['compute_node_instance_type']

### Region

Just run a check here to make sure that there is a valid region.

In [132]:
#available_regions = get_regions()

# London
# region = 'eu-west-2'
os.environ["AWS_DEFAULT_REGION"] = CONFIG['aws_region']

pcluster_config.region = region

### EC2 Key Pairs

In [133]:
ec2_key_pairs = easyconfig._get_keys()
ec2_key_pairs

['pfaffmanager_key']

### Networking

In [140]:
# Validation
# This would be a very good check to do
#azs_for_head_node_type = get_supported_az_for_one_instance_type(cluster_config.head_node_instance_type)
#azs_for_compute_type = get_supported_az_for_one_instance_type(cluster_config.compute_instance_type)
#common_availability_zones = set(azs_for_head_node_type) & set(azs_for_compute_type)

In [158]:
vpcs_and_subnets = easyconfig._get_vpcs_and_subnets()
#vpcs_and_subnets
#vpc_id = vpcs_and_subnets['vpc_list'][0]['id']
vpc_id = CONFIG['vpc_id']

In [159]:
vpc_parameters["vpc_id"] = vpc_id
subnet_list = vpcs_and_subnets["vpc_subnets"][vpc_id]
#subnet_list

### Automate Subnet Creation - IMPORTANT

This is the main step in this module. PCluster is real picky about it's subnets so we make sure to use the bootstrap.

In [152]:
min_subnet_size = int(cluster_config.max_cluster_size)
networking_configuration = PublicPrivateNetworkConfig()
#subnets = automate_subnet_creation(vpc_id, networking_configuration, min_subnet_size)

In [162]:
vpcs_and_subnets = easyconfig._get_vpcs_and_subnets()
subnet_list = vpcs_and_subnets["vpc_subnets"][vpc_id]
#subnet_list

In [157]:
def get_private_subnet(subnet_list):
    for subnet in subnet_list:
        if subnet['name'] == 'ParallelClusterPrivateSubnet':
            return subnet
        
def get_public_subnet(subnet_list):
    for subnet in subnet_list:
        if subnet['name'] == 'ParallelClusterPublicSubnet':
            return subnet
        
private_subnet = get_private_subnet(subnet_list)
public_subnet = get_public_subnet(subnet_list)

OrderedDict([('id', 'subnet-0e283ed98f1b6b0d2'), ('name', 'ParallelClusterPrivateSubnet'), ('size', 4096), ('availability_zone', 'us-east-1d')])
OrderedDict([('id', 'subnet-0322108492fd84a41'), ('name', 'ParallelClusterPublicSubnet'), ('size', 256), ('availability_zone', 'us-east-1b')])


In [207]:
master_subnet_id = public_subnet['id']
compute_subnet_id = private_subnet['id']
CONFIG['master_subnet_id'] = master_subnet_id
CONFIG['compute_subnet_id'] = compute_subnet_id

## Scheduler

In [208]:
# For now only supporting slurm, but this may be good to have
# Validators for web ui
schedulers = get_supported_schedulers()
supported_oses = get_supported_os_for_scheduler(scheduler)

In [209]:
cluster_config = ClusterConfigureHelper(cluster_section, scheduler)
cluster_config

In [210]:
# This will go in the Plugin and Web UI
#supported_instance_types_head = get_supported_instance_types()
#supported_instance_types_compute = get_supported_compute_instance_types(scheduler)
#supported_instance_types_head.sort()
#supported_instance_types_compute.sort()

## Queues and Compute Resources

The format is [compute_resource <compute-resource-name>]. compute-resource-name must start with a letter, contain no more than 30 characters, and only contain letters, numbers, hyphens (-), and underscores (_).
    
The format is [queue <queue-name>]. queue-name must start with a lowercase letter, contain no more than 30 characters, and only contain lowercase letters, numbers, and hyphens (-).

At any time, there can be between zero (0) and the max number of dynamic nodes in a [compute_resource].
    
compute_resource_settings

(Required) Identifies the [compute_resource] sections containing the compute resources configurations for this queue. The section names must start with a letter, contain no more than 30 characters, and only contain letters, numbers, hyphens (-), and underscores (_).

*Up to three (3) [compute_resource] sections are supported for each [queue] section.*

In [211]:
## Clean up queue and compute resources
## TODO Write Validator that - contain no more than 30 characters, and only contain letters, numbers, hyphens (-), and underscores (_).

for compute_resource in CONFIG['compute_resources']:
    compute_resource['name'] = compute_resource['instance_type'].replace('.', '-')


queue_settings = []
for queue in CONFIG['queues']:
    queue_settings.append(queue['name'])
    instance_types_cleaned = []
    for instance_type in queue['compute_resource_instance_types']:
        instance_types_cleaned.append(instance_type.replace('.', '-'))
        

    queue['compute_resource_settings'] = ','.join(instance_types_cleaned)
    print(queue)
    
CONFIG['queue_settings'] = ','.join(queue_settings)

{'name': 'dev', 'enable_efa': False, 'enable_efa_gdr': False, 'compute_resource_instance_types': ['t3a.medium', 't3a.2xlarge'], 'compute_resource_settings': 't3a-medium,t3a-2xlarge'}
{'name': 'cpu', 'enable_efa': False, 'enable_efa_gdr': False, 'compute_resource_instance_types': ['t3a.medium', 't3a.2xlarge'], 'compute_resource_settings': 't3a-medium,t3a-2xlarge'}
{'name': 'gpu', 'enable_efa': False, 'enable_efa_gdr': False, 'compute_resource_instance_types': ['t3a.medium', 't3a.2xlarge'], 'compute_resource_settings': 't3a-medium,t3a-2xlarge'}


In [99]:
data = {'cookiecutter': CONFIG}
rtemplate = Environment(loader=BaseLoader).from_string(pcluster_template)
rendered_template = rtemplate.render(**data)
print(rendered_template)

NameError: name 'pcluster_template' is not defined

## Get the Custom AMI ID

We're using Alinux2, and we need to get the correct AMI id for the region.

In [75]:
#AMIS_LIST
r = requests.get(AMIS_LIST)
amis_list = r.content.decode('UTF-8')
amis_list = amis_list.split('\n')
amis_list[0:10]

['## x86_64',
 '# alinux',
 'af-south-1: ami-0f2e2135a05f814df',
 'ap-east-1: ami-05ef7cb79d3a43092',
 'ap-northeast-1: ami-006b7f4c929aaf4a9',
 'ap-northeast-2: ami-06a49824cc501a981',
 'ap-northeast-3: UNSUPPORTED',
 'ap-south-1: ami-0a47fd68cf7034c58',
 'ap-southeast-1: ami-0e6cfdde386164836',
 'ap-southeast-2: ami-0ba7f788162c4a4de']

In [80]:
alinux2_list = {}

seen_x86_64 = 0
seen_alinux2 = 0
for line in amis_list:
    if '## ' in line:
        seen_x86_64 = 0
        if 'x86_64' in line:
            seen_x86_64 = 1
    if '# ' in line:
        seen_alinux2=0
        if 'alinux2' in line:
            seen_alinux2=1
    else:
        if seen_alinux2 and seen_x86_64:
            t = line.split(': ')
            t[0] = t[0].strip()
            t[1] = t[1].strip()
            alinux2_list[t[0]] = t[1]
            
alinux2_list

{'af-south-1': 'ami-046f49b550ce90d8a',
 'ap-east-1': 'ami-0ec0d099b8a276aec',
 'ap-northeast-1': 'ami-0a13402dc88c19be2',
 'ap-northeast-2': 'ami-0bdfbd3521caa5dd2',
 'ap-northeast-3': 'UNSUPPORTED',
 'ap-south-1': 'ami-0059d599d21636768',
 'ap-southeast-1': 'ami-074f58cccc7ebb68f',
 'ap-southeast-2': 'ami-04b4a20ee9f67608f',
 'ca-central-1': 'ami-0523a9bc4151ee96e',
 'cn-north-1': 'ami-0a12307d2d0ddc535',
 'cn-northwest-1': 'ami-0ef379c7fd5eb332e',
 'eu-central-1': 'ami-07055c21834b0bc56',
 'eu-north-1': 'ami-0fe475ca307943eb1',
 'eu-south-1': 'ami-042fc69d71433a75c',
 'eu-west-1': 'ami-063ac3df7f8595751',
 'eu-west-2': 'ami-086111e12527fa455',
 'eu-west-3': 'ami-0258a5a8320ccfa42',
 'me-south-1': 'ami-0c98692d98eb38c50',
 'sa-east-1': 'ami-0f463bcf6d86cad85',
 'us-east-1': 'ami-0b71488efbe422723',
 'us-east-2': 'ami-0075df3faa5b6e07e',
 'us-gov-east-1': 'ami-057f7c2d5a1ca7b7d',
 'us-gov-west-1': 'ami-01222b796bafd609f',
 'us-west-1': 'ami-01c4b0b6d5597b80b',
 'us-west-2': 'ami-079fa

In [77]:
alinux2_list[CONFIG['aws_region']]

'ami-0b1f998cf2b1498db'

In [78]:
AMIS_LIST

'https://raw.githubusercontent.com/aws/aws-parallelcluster/v2.10.1/amis.txt'

In [48]:
CONFIG['custom_ami_id'] = alinux2_list[CONFIG['aws_region']]

## Cleanup

In [49]:
#os.remove(tconfig.name)

In [60]:
CONFIG['custom_ami_id'] = alinux2_list[CONFIG['aws_region']]
with open('/home/jovyan/TEST_APP/cookiecutter.json', 'w') as outfile:
    json.dump(CONFIG, outfile, indent=4, sort_keys=True)

In [85]:
import boto3

In [104]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
account_id

'858286506743'

In [88]:
client = boto3.client('cloudformation')

In [90]:
client.list_exports()

{'Exports': [],
 'ResponseMetadata': {'RequestId': '7108a5c7-b22a-4a32-bd7f-5b2d092370d6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7108a5c7-b22a-4a32-bd7f-5b2d092370d6',
   'content-type': 'text/xml',
   'content-length': '272',
   'date': 'Wed, 27 Jan 2021 12:19:23 GMT'},
  'RetryAttempts': 0}}

In [112]:
stacks = client.describe_stacks(StackName='parallelcluster-slurm-cluster-20210127-development',)

In [116]:
len(stacks['Stacks'])

1

In [124]:
stacks['Stacks'][0].keys()

dict_keys(['StackId', 'StackName', 'Description', 'Parameters', 'CreationTime', 'RollbackConfiguration', 'StackStatus', 'DisableRollback', 'NotificationARNs', 'Capabilities', 'Outputs', 'Tags', 'EnableTerminationProtection', 'DriftInformation'])

In [122]:
for parameter in stacks['Stacks'][0]['Parameters']:
    key = parameter['ParameterKey']
    if 'Master' in key:
        print('ParameterKey: {}'.format(parameter['ParameterKey']))
    

ParameterKey: MasterSubnetId
ParameterKey: MasterRootVolumeSize
ParameterKey: MasterInstanceType


In [125]:
for output in stacks['Stacks'][0]['Outputs']:
    print(output)
    #key = parameter['ParameterKey']
    #if 'Master' in key:
    #    print('ParameterKey: {}'.format(parameter['ParameterKey']))

{'OutputKey': 'ArtifactS3RootDirectory', 'OutputValue': 'parallelcluster-slurm-cluster-20210127-develop-28jog97ymlhz3z72', 'Description': 'Root directory in S3 bucket where cluster artifacts are stored'}
{'OutputKey': 'IsHITCluster', 'OutputValue': 'true'}
{'OutputKey': 'ClusterUser', 'OutputValue': 'ec2-user', 'Description': 'Username to login to head node'}
{'OutputKey': 'MasterPrivateIP', 'OutputValue': '172.31.97.236', 'Description': 'Private IP Address of the head node'}
{'OutputKey': 'ResourcesS3Bucket', 'OutputValue': 'parallelcluster-p4b4uzx457i1111h', 'Description': 'S3 user bucket where AWS ParallelCluster resources are stored'}
{'OutputKey': 'ClusterConfigMetadata', 'OutputValue': '{"sections": {"cluster": ["default"], "scaling": ["default"], "vpc": ["default"]}}'}
{'OutputKey': 'GangliaPrivateURL', 'OutputValue': 'http://172.31.97.236/ganglia/', 'Description': 'Private URL to access Ganglia (disabled by default)'}


In [129]:
cfn = boto3.client('cloudformation')
my_stack_name='parallelcluster-slurm-cluster-20210127-development'
list(map(lambda x: cfn.describe_stack_resources(StackName=x['PhysicalResourceId'])['StackResources'], cfn.describe_stack_resources(StackName=my_stack_name)['StackResources']))

ClientError: An error occurred (ValidationError) when calling the DescribeStackResources operation: Stack with id pcluster-CleanupResources-872a7680-6096-11eb-8ea4-1266e578c117 does not exist

In [131]:
stack_resources = cfn.describe_stack_resources(StackName=my_stack_name)['StackResources']

In [132]:
stack_resources[0]

{'StackName': 'parallelcluster-slurm-cluster-20210127-development',
 'StackId': 'arn:aws:cloudformation:us-east-1:858286506743:stack/parallelcluster-slurm-cluster-20210127-development/872a7680-6096-11eb-8ea4-1266e578c117',
 'LogicalResourceId': 'CleanupResourcesFunction',
 'PhysicalResourceId': 'pcluster-CleanupResources-872a7680-6096-11eb-8ea4-1266e578c117',
 'ResourceType': 'AWS::Lambda::Function',
 'Timestamp': datetime.datetime(2021, 1, 27, 11, 55, 47, 332000, tzinfo=tzlocal()),
 'ResourceStatus': 'CREATE_COMPLETE',
 'DriftInformation': {'StackResourceDriftStatus': 'NOT_CHECKED'}}